In [1]:
from audiocraft.models import MusicGen
from audiocraft.utils.autocast import TorchAutocast
import torch
from torch import nn
import os

folder = './musicgen-stereo-small'
os.makedirs(folder, exist_ok=True)
prompt = '80s pop track with bassy drums and synth'

cfg = 5
temperature = 0.7
top_k = 500
top_p = 0.0
max_len = 256

/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = MusicGen.get_pretrained("facebook/musicgen-stereo-small", device='cpu')

/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


Extractions of the model:

In [3]:
wav = model.generate([prompt])

[ConditioningAttributes(text={'description': '80s pop track with bassy drums and synth'}, wav={'self_wav': WavCondition(wav=tensor([[[0.]]]), length=tensor([0]), sample_rate=[32000], path=[None], seek_time=[])}, joint_embed={})]
torch.Size([2, 12, 768])
torch.Size([1024, 768])
yeet
tensor([[[-0.2954,  0.0587, -0.0238,  ...,  0.0086, -0.0069, -0.0077]],

        [[-0.2954,  0.0587, -0.0238,  ...,  0.0086, -0.0069, -0.0077]]])
{'description': (tensor([[[ 0.3144,  0.1624,  0.1167,  ...,  0.4033,  0.3492,  0.5680],
         [-0.0924,  0.2206,  0.6471,  ..., -0.2670,  0.6927,  1.0012],
         [-0.2491, -0.4061,  0.0845,  ..., -0.2199, -0.7242,  0.6362],
         ...,
         [ 0.1737,  0.3411, -0.2478,  ...,  0.1705,  0.1207,  0.0260],
         [ 0.4187,  0.2239,  0.1036,  ...,  0.1287,  0.2915, -0.6879],
         [-0.0094, -0.0076, -0.0243,  ...,  0.0028, -0.0176, -0.0108]],

        [[-0.0000,  0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
         [-0.0000,  0.0000, -0.0000,  ...

RuntimeError: No active exception to reraise

In [ ]:
from IPython.display import Audio

sampling_rate = 32000
Audio(wav[0].detach().numpy(), rate=sampling_rate)

Unfolding the Streaming Transformer to just the regular

In [ ]:
model.lm.transformer

StreamingTransformer(
  (layers): ModuleList(
    (0-23): 24 x StreamingTransformerLayer(
      (self_attn): StreamingMultiheadAttention(
        (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
      )
      (linear1): Linear(in_features=1024, out_features=4096, bias=False)
      (dropout): Dropout(p=0.0, inplace=False)
      (linear2): Linear(in_features=4096, out_features=1024, bias=False)
      (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.0, inplace=False)
      (dropout2): Dropout(p=0.0, inplace=False)
      (layer_scale_1): Identity()
      (layer_scale_2): Identity()
      (cross_attention): StreamingMultiheadAttention(
        (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
      )
      (dropout_cross): Dropout(p=0.0, inplace=False)
      (norm_cross): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (layer_sc

In [5]:
from audiocraft.modules.transformer import StreamingTransformerLayer, create_sin_embedding

class UnwrappedTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        num_layers = 24

        self.positional_embedding = 'sin'
        self.max_period = 10000
        self.positional_scale = 1.0
        self.weight_decay = None
        self.lr = None
        self.rope = None
        self.checkpointing = 'none'

        self.layers = nn.ModuleList()
        for idx in range(num_layers):
            self.layers.append(
                StreamingTransformerLayer(
                    d_model=1024, num_heads=16, dim_feedforward=4096,
                    dropout=0.0, bias_ff=False, bias_attn=False,
                    causal=True, past_context=None, custom=False,
                    memory_efficient=True, attention_as_float32=False,
                    cross_attention=True, layer_scale=None, rope=self.rope,
                    device='cpu', dtype=torch.float32, **{'norm': 'layer_norm', 'norm_first': True, 'activation': 'gelu', 'qk_layer_norm': False, 'qk_layer_norm_cross': False, 'attention_dropout': None, 'kv_repeat': 1})
                )

    def forward(self, x: torch.Tensor, offsets, *args, **kwargs):
        B, T, C = x.shape

        if self.positional_embedding in ['sin', 'sin_rope']:
            positions = torch.arange(T, device=x.device).view(1, -1, 1)
            positions = positions + offsets.view(-1, 1, 1)
            pos_emb = create_sin_embedding(positions, C, max_period=self.max_period, dtype=x.dtype)
            x = x + self.positional_scale * pos_emb

        for layer in self.layers:
            x = layer(x, *args, **kwargs)

        offsets = offsets + T

        return x, offsets

In [14]:
from collections import namedtuple
from audiocraft.modules.codebooks_patterns import Pattern

projection_layer = model.lm.condition_provider.conditioners.description.output_proj # Take the embed enlargement layer
embbedding_layer = model.lm.emb
fuser = model.lm.fuser
transformer = UnwrappedTransformer()
transformer.load_state_dict(model.lm.transformer.state_dict())

class LMModelWrapper(nn.Module):
    def __init__(self, projection_layer, emb, fuser, transformer): #, lm_model, projection_layer):
        super().__init__()
        # Layers
        self.projection = projection_layer
        self.emb = emb
        self.fuser = fuser
        self.transformer = transformer

        #Vars
        self.num_codebooks = 8
        self.audio_channels = 2

    def triu_onnx(x, diagonal=0):
        l, w = x.shape
        arange_rows = torch.arange(l, device=x.device)

        arange_cols = torch.arange(w, device=x.device)
        mask = arange_cols.expand(l, w)

        arange_rows = arange_rows[:, None] + diagonal
        mask = mask >= arange_rows
        return x.masked_fill(mask == 0, 0)

    def build_delay_pattern_mask(self, input_ids, pad_token_id, max_length):
        input_ids = input_ids.reshape(-1, self.num_codebooks, input_ids.shape[-1])
        bsz, num_codebooks, seq_len = input_ids.shape

        max_length = max_length.unsqueeze(0)  # Convert to tensor for ONNX
        condition = (max_length > 0).long()  # Create a condition tensor

        input_ids_shifted = condition * (
            torch.ones((bsz, num_codebooks, max_length.item()), dtype=torch.long, device=input_ids.device) * -1
        )

        max_length = max_length.item()

        channel_codebooks = num_codebooks // 2 if self.audio_channels == 2 else num_codebooks
        # we only apply the mask if we have a large enough seq len - otherwise we return as is
        if max_length < 2 * channel_codebooks - 1:
            return input_ids.reshape(bsz * num_codebooks, -1), input_ids_shifted.reshape(bsz * num_codebooks, -1)

        # fill the shifted ids with the prompt entries, offset by the codebook idx
        for codebook in range(channel_codebooks):
            if self.audio_channels == 1:
                # mono channel - loop over the codebooks one-by-one
                input_ids_shifted[:, codebook, codebook : seq_len + codebook] = input_ids[:, codebook]
            else:
                # left/right channels are interleaved in the generated codebooks, so handle one then the other
                input_ids_shifted[:, 2 * codebook, codebook : seq_len + codebook] = input_ids[:, 2 * codebook]
                input_ids_shifted[:, 2 * codebook + 1, codebook : seq_len + codebook] = input_ids[:, 2 * codebook + 1]
        # construct a pattern mask that indicates the positions of padding tokens for each codebook
        # first fill the upper triangular part (the EOS padding)
        delay_pattern = torch.triu(
            torch.ones((channel_codebooks, max_length), dtype=torch.bool), diagonal=max_length - channel_codebooks + 1
        ).to(torch.int64)
        # then fill the lower triangular part (the BOS padding)
        delay_pattern = delay_pattern + torch.tril(torch.ones((channel_codebooks, max_length), dtype=torch.int64))

        if self.audio_channels == 2:
            # for left/right channel we need to duplicate every row of the pattern mask in an interleaved fashion
            delay_pattern = delay_pattern.repeat_interleave(2, dim=0)

        delay_pattern = delay_pattern.to(torch.bool)

        mask = ~delay_pattern.to(input_ids.device)
        input_ids = mask * input_ids_shifted + ~mask * pad_token_id

        # find the first position to start generating - this is the first place we have the -1 token
        # and will always be in the first codebook (since it has no codebook offset)
        first_codebook_ids = input_ids[:, 0, :]
        start_ids = (first_codebook_ids == -1).nonzero()[:, 1]
        if len(start_ids) > 0:
            first_start_id = min(start_ids)
        else:
            # we have no tokens that need to be filled - return entire matrix of input ids
            first_start_id = seq_len

        # (bsz * num_codebooks, seq_len) -> (bsz, num_codebooks, seq_len)
        pattern_mask = input_ids.reshape(bsz * num_codebooks, -1)
        input_ids = input_ids[..., :first_start_id].reshape(bsz * num_codebooks, -1)
        
        return input_ids, pattern_mask 

    def forward(
            self,
            embeddings: torch.Tensor,           # From the T5 Model
            attention_mask: torch.Tensor,       # From the Tokenizer
            tokens: torch.Tensor,               # n_samples, codebooks, tokens
        ):
        cfg = torch.tensor([5])
        max_length = torch.tensor([256])
        embeddings = self.projection(embeddings.to(self.projection.weight))
        embeddings = (embeddings * attention_mask.unsqueeze(-1))

        num_samples, num_codebooks = tokens.size(0)-1, 8 # Find a better way to extract this

        input_ids = torch.tensor([[2048]] * self.num_codebooks)
        input_ids, pattern = self.build_delay_pattern_mask(input_ids=input_ids, pad_token_id=2048, max_length=max_length)
        mask = torch.where(pattern == -1, 1, 0)

        prev_offset = 0
        offset_transformer = torch.zeros(tokens.size(0), dtype=torch.long)

        for offset in range(1, max_length):
            curr_sequence = pattern[..., prev_offset : offset].unsqueeze(0)
            curr_mask = mask[None, ..., prev_offset:offset].expand(num_samples, -1, -1)

            # Similar to the _sample_next_token fn
            sequence = torch.cat([curr_sequence, curr_sequence], dim=0)


            # Go into the forward of the LM model
            B, K, S = sequence.shape
            input_ = sum([self.emb[k](sequence[:, k]) for k in range(K)])
            input_, cross_attention_input = self.fuser(input_, {'description': (embeddings, attention_mask)})

            out, offset_transformer = self.transformer(input_, offset_transformer, cross_attention_src=cross_attention_input, src_mask=(None))
            print(out)
            print()
            break
            
            #Not perse true
            prev_offset+=1

        return input_, embeddings, attention_mask, tokens


        print(sequence)

        

        pass
lm_model = LMModelWrapper(projection_layer, embbedding_layer, fuser, transformer)
lm_model(tokens=inputs['input_ids'], embeddings=torch.tensor(embeded), attention_mask = inputs['attention_mask'])

tensor([[[1.1435, 0.8727, 0.0470,  ..., 1.8955, 0.3980, 0.5159]],

        [[1.2410, 0.8026, 0.1755,  ..., 1.9652, 0.2593, 0.6302]]],
       grad_fn=<AddBackward0>)



(tensor([[[-0.2954,  0.0587, -0.0238,  ...,  0.0086, -0.0069, -0.0077]],
 
         [[-0.2954,  0.0587, -0.0238,  ...,  0.0086, -0.0069, -0.0077]]],
        grad_fn=<AddBackward0>),
 tensor([[[ 0.3144,  0.1624,  0.1167,  ...,  0.4033,  0.3492,  0.5680],
          [-0.0924,  0.2206,  0.6471,  ..., -0.2670,  0.6927,  1.0012],
          [-0.2491, -0.4061,  0.0845,  ..., -0.2199, -0.7242,  0.6362],
          ...,
          [ 0.1737,  0.3411, -0.2478,  ...,  0.1705,  0.1207,  0.0260],
          [ 0.4187,  0.2239,  0.1036,  ...,  0.1287,  0.2915, -0.6879],
          [-0.0094, -0.0076, -0.0243,  ...,  0.0028, -0.0176, -0.0108]],
 
         [[-0.0000,  0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0000, -0.0000,  ...,  0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000,  ...,  0.0000, -0.0000, -0.0000],
          ...,
          [-0.0000,  0.0000, -0.0000,  ...,  0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000,  ...,  0.0000, -0.0000, -0.

In [17]:
# Define the dynamic axes for variable-length input shapes
dynamic_axes = {
    'embeddings': {0: 'batch_size', 1: 'sequence_length'},  # Allow variable batch size and sequence length
    'attention_mask': {0: 'batch_size', 1: 'sequence_length'},  # Output will also have variable batch size and sequence length
    'tokens': {0: 'batch_size', 1: 'sequence_length'}  # Allow variable batch size and sequence length
}

# Example input shapes (with batch size = 2, sequence length = 10)
dummy_tokens = inputs['input_ids']
dummy_embeddings = torch.tensor(embeded, dtype=torch.float32)
dummy_attention_mask = inputs['attention_mask'].to(torch.float32)

# Export the model to ONNX format
torch.onnx.export(
    lm_model,
    (dummy_embeddings, dummy_attention_mask, dummy_tokens),
    f"{folder}/test.onnx",
    input_names=['embeddings', 'attention_mask', 'tokens'],
    output_names=['input_', 'embeddings', 'attention_mask', 'tokens'],
    dynamic_axes=dynamic_axes,
    opset_version=17
)

/tmp/ipykernel_2546311/1139425351.py:100: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  cfg = torch.tensor([5])
/tmp/ipykernel_2546311/1139425351.py:101: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  max_length = torch.tensor([256])
/tmp/ipykernel_2546311/1139425351.py:107: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same ev

tensor([[[1.1435, 0.8727, 0.0470,  ..., 1.8955, 0.3980, 0.5159]],

        [[1.2410, 0.8026, 0.1755,  ..., 1.9652, 0.2593, 0.6302]]],
       grad_fn=<AddBackward0>)



RuntimeError: outerNode->outputs().size() == node->inputs().size() INTERNAL ASSERT FAILED at "../torch/csrc/jit/passes/dead_code_elimination.cpp":140, please report a bug to PyTorch. 

### The encoder (Just using the T5 Encoder):
optimum-cli export onnx --model google-t5/t5-base T5Base

In [8]:
inputs = model.lm.condition_provider.conditioners.description.tokenize([prompt, None])

In [9]:
import onnxruntime as ort

ort_session = ort.InferenceSession('./T5Base/encoder_model.onnx')

ort_inputs = {
    k:v.detach().numpy() for k,v in inputs.items()
}

embeded = ort_session.run(None, ort_inputs)[0]

In [ ]:
import onnxruntime as ort

ort_session = ort.InferenceSession(f'{folder}/test.onnx')

for input_meta in ort_session.get_inputs():
    print(f"Input Name: {input_meta.name}")
    print(f"Shape: {input_meta.shape}")
    print(f"Type: {input_meta.type}")

Input Name: embeddings.1
Shape: ['embeddings.1_dim_0', 'embeddings.1_dim_1', 768]
Type: tensor(float)
Input Name: attention_mask.1
Shape: ['embeddings.1_dim_0', 'embeddings.1_dim_1']
Type: tensor(float)
Input Name: tokens.1
Shape: ['embeddings.1_dim_0', 'embeddings.1_dim_1']
Type: tensor(int64)


In [ ]:
import onnxruntime as ort
import numpy as np

ort_session = ort.InferenceSession(f'{folder}/test.onnx')

ort_inputs = {
    'embeddings.1': embeded,
    'attention_mask.1': inputs['attention_mask'].detach().numpy().astype(np.float32),
    'tokens.1': inputs['input_ids'].detach().numpy()
}

embeded = ort_session.run(None, ort_inputs)

In [ ]:
embeded

[array([[[-0.29537773,  0.05873203, -0.02379608, ...,  0.00863934,
          -0.00691605, -0.00769234]],
 
        [[-0.29537773,  0.05873203, -0.02379608, ...,  0.00863934,
          -0.00691605, -0.00769234]]], dtype=float32),
 array([[[ 0.31444335,  0.16241327,  0.11668988, ...,  0.40329838,
           0.34918803,  0.5680022 ],
         [-0.09240554,  0.22058195,  0.64709044, ..., -0.2670372 ,
           0.6926601 ,  1.001211  ],
         [-0.24913806, -0.40614206,  0.08446971, ..., -0.21988793,
          -0.7242352 ,  0.63619673],
         ...,
         [ 0.17368768,  0.34110174, -0.2478149 , ...,  0.17052141,
           0.12066975,  0.02595249],
         [ 0.4187473 ,  0.22387516,  0.10361063, ...,  0.12866525,
           0.2915484 , -0.68793195],
         [-0.00938433, -0.00760569, -0.02425367, ...,  0.00280249,
          -0.01758854, -0.0108365 ]],
 
        [[-0.        ,  0.        , -0.        , ..., -0.        ,
          -0.        ,  0.        ],
         [-0.        ,  0.